# Create Mini Molecule Set from Large Database

Goal 

1) Try to include as many parameter types from smirnoff99Frosst as possible

2) Try to include all parm@Frosst atomtypes from initial set

2) Limit the number of total molecules 

**Authors**:
* Caitlin C. Bannan (UCI)
* Followed *parm@frossty-y to SMIRNOFF* script by  David L. Mobley (UCI) as example on how to type molecules with smirnoff99Frosst


In [1]:
# Imports
from __future__ import print_function
from openeye.oechem import *
from openeye.oeiupac import *
from openeye.oeomega import *
from openeye.oedepict import *
from IPython.display import display
from openforcefield.typing.engines.smirnoff.forcefield import *
from openforcefield.typing.engines.smirnoff.forcefield_utils import get_molecule_parameterIDs
from openforcefield.utils import *
% matplotlib inline
import matplotlib
import numpy as np
import pylab as pl
import matplotlib.pyplot as plt
import time
import IPython
import pickle
import copy

## 1. Load Molecules

Load all DrugBank.sdf files

In [2]:
inf = "DrugBank_atyped.oeb"
starting_molecules = list()
init = time.time()
ifs = oemolistream(inf)
mol = OEMol()
while OEReadMolecule(ifs,mol):
    starting_molecules.append(OEMol(mol))
ifs.close()
end = time.time()
print("Found %i molecules in %s \n it took %.1f seconds" % (len(starting_molecules), inf, end-init))

Found 7133 molecules in DrugBank_atyped.oeb 
 it took 1.0 seconds


## 1.5 Remove unwanted molecules

Generally speaking this removes molecules smirnoff99Frosst cannot currently type. 
As this notebook was created to generate a molecule set for smarty/smirky tests it also removes molecules the parm@Frosst forcefield cannot type

* No metals
* No more than 100 heavy atoms
* Grater than 2 heavy atoms
* No gg atom types (that is the generic parm@Frosst) atomtype

In [3]:
def hasAtomType(mol, typ):
    for atom in mol.GetAtoms():
        if atom.GetType() in typ:
            return True
    return False
    
ff = ForceField('forcefield/smirnoff99Frosst.ffxml')  
labels = ff.labelMolecules(starting_molecules)
molecules = list()
for index, mol in enumerate(starting_molecules):
    if OECount(mol, OEIsMetal()) > 0:
        continue
    if OECount(mol, OEIsHeavy()) > 100:
        continue
    if OECount(mol, OEIsHeavy()) < 4:
        continue
    if hasAtomType(mol, ['gg','IM']):
        continue
    pids = [pid for force, lists in labels[index].items() for (indices,pid, smarts) in lists]
    if 'n1' in pids:
        continue
    if 'b1' in pids:
        continue
    if 'a1' in pids:
        continue
    if 't1' in pids:
        continue
    molecules.append(OEMol(mol))

print("Keeping %i molecules of the original %i" % (len(molecules), len(starting_molecules)))

Keeping 6546 molecules of the original 7133


## 2. Load SMIRNOFF99Frosst and label molecules

In [4]:
init = time.time()
labels = ff.labelMolecules(molecules)
end = time.time()
print("Took %.1f seconds to label all %i molecules" % (end-init, len(molecules)))

Took 762.0 seconds to label all 6759 molecules


## 3. Store initial information

In [7]:
def getInitialInformation(labels, molecules):
    keep_mols = set()
    unused_pids = set()
    dict_mols = dict()
    for i in range(len(labels)):
        dict_mols[i] = set()
    dict_pid = dict()
    # loop through molecules:
    for idx, molDict in enumerate(labels):
        # loop through forcetypes from smirnoff99Frosst label
        for force, idList in molDict.items():
            # loop through pid lists
            for (idices, pid, smirks) in idList:
                if not pid in dict_pid:
                    dict_pid[pid] = set()
                unused_pids.add(pid)
                dict_mols[idx].add(pid)
                dict_pid[pid].add(idx)
        
        # Loop through atoms in molecule treating where
        # atomtypes are the parameter id
        mol = molecules[idx]
        for atom in mol.GetAtoms():
            typ = atom.GetType()
            if not typ in dict_pid:
                dict_pid[typ] = set()
            unused_pids.add(typ)
            dict_mols[idx].add(typ)
            dict_pid[typ].add(idx)
                
    return unused_pids, dict_mols, dict_pid

In [8]:
init = time.time()
unused_pids, dict_mols, dict_pid = getInitialInformation(labels, molecules)
all_pids = list(unused_pids)
end = time.time()
print("Created initial dictionaries and lists in %.1f seconds" % (end-init))

Created initial dictionaries and lists in 2.9 seconds


## 4. Remove unnecessary molecules

In [9]:
# store molecules (to keep) and pids (to remove)
init = time.time()
keep_mols = set()

length = 2
while len(unused_pids) > 0:
    # Track if a change has been made
    to_remove_pids = set()
    unchanged = True
    
    # For all pids still in the unused list
    for pid in unused_pids:    
        # get the list of molecules with that parameter
        molList = copy.copy(dict_pid[pid])
        
        # Trying to remove the limited number of molecules, so 
        # start by only considering pids with in a small number of molecules, 
        # if no changes are made then the length is increased 
        if len(molList) < length:
            unchanged = False
            for m in molList:
                keep_mols.add(m) # Keep this molecule
                for pid in dict_mols[m]:
                    # remove all pids in this molecule from the unused list
                    to_remove_pids.add(pid) 
                if m in dict_pid[pid]:
                    dict_pid[pid].remove(m) # remove this molecule from pid dict as it has already been stored
            
    # updated unused_pids
    for pid in to_remove_pids:
        if pid in unused_pids:
            unused_pids.remove(pid)
    
    # update length of molList considered
    if unchanged:
        length += 1
    # If A change was made reset length to 2
    else:
        print("the length was %i" % length)
        length = 2

end = time.time()
print("Removing unnecessary molecules took %.1f seconds" % (end-init))

the length was 2
the length was 3
the length was 4
the length was 5
the length was 6
the length was 7
the length was 8
the length was 9
the length was 13
the length was 15
the length was 16
the length was 20
the length was 33
the length was 42
Removing unnecessary molecules took 0.0 seconds


In [10]:

# Make an updated list with only the stored molecules
new_molecules = list()
for idx in keep_mols:
    new_molecules.append(OEMol(molecules[idx]))
print("There are %i molecules in the final set" % len(new_molecules))

There are 269 molecules in the final set


## 5. Get information about new molecules

In [11]:
init = time.time()
new_labels = [labels[idx] for idx in keep_mols]
new_unused_pids, new_dict_mols, new_dict_pid = getInitialInformation(new_labels, new_molecules)

for pid in all_pids:
    if pid not in new_unused_pids:
        print("pid %s is missing in new_molecules" % pid)
end = time.time()
print("checking and storing new molecules took %.1f seconds" % (end-init))

checking and storing new molecules took 0.1 seconds


In [12]:
def getAtomtypeCounts(molecules):
    A = dict()
    for mol in new_molecules:
        for atom in mol.GetAtoms():
            a_num = atom.GetAtomicNum()
            a_type = atom.GetType()
            if not a_num in A:
                A[a_num] = set()
            A[a_num].add(a_type)
    return A

In [13]:
print( len(all_pids))
n = [a for a in all_pids if a[0]=='n']
print("Vdw %i" % len(n))
b = [a for a in all_pids if a[0]=='b']
print("Bonds %i" % len(b))
aS = [a for a in all_pids if a[0]=='a']
print("Angles %i" % len(aS))
t = [a for a in all_pids if a[0]=='t']
print("Torsions %i" % len(t))

oldA = getAtomtypeCounts(molecules)
newA = getAtomtypeCounts(new_molecules)
print("%20s %20s %20s" % ('AtomicNum', 'in Old', 'in New'))
for k, e in oldA.items():
    new_e = newA[k]
    print("%20s %20s %20s" % (k, len(e), len(new_e)))

319
Vdw 25
Bonds 73
Angles 33
Torsions 137
           AtomicNum               in Old               in New
                   6                   12                   12
                   7                    8                    8
                   8                    5                    5
                   1                   12                   12
                  16                    5                    5
                   9                    1                    1
                  15                    1                    1
                  17                    1                    1
                  35                    1                    1
                  53                    1                    1


## 6. Store new Molecules

In [14]:
# Get 3D coordinates
omega = OEOmega()
omega.SetMaxConfs(1)
omega.SetIncludeInput(False)
omega.SetCanonOrder(False)
omega.SetSampleHydrogens(True)
omega.SetStrictStereo(True)
omega.SetStrictAtomTypes(True)

for mol in new_molecules[209:]:
    OEAddExplicitHydrogens(mol)
    omega(mol)

In [15]:
# Following example in Christopher Bayly's oeb-to-FF-and-tripos-mol2.py script
# Make a parm@Frosst atomtype mol2 file
ofsff = oechem.oemolostream()
ofsff.SetFlavor( oechem.OEFormat_MOL2, oechem.OEOFlavor_MOL2_Forcefield )
ofsff.open("MiniDrugBank_ff.mol2")
# and Tripos atomtype mol2 file
ofsTri = oechem.oemolostream()
ofsTri.SetFlavor( oechem.OEFormat_MOL2, oechem.OEOFlavor_MOL2_Forcefield )
ofsTri.open('MiniDrugBank_tripos.mol2')

for index, c_mol in enumerate(new_molecules):
    c_mol.SetTitle("MiniDrugBank_%i" % index)
    mol1 = OEMol(c_mol)
    oechem.OETriposAtomNames(mol1)
    oechem.OEWriteConstMolecule(ofsff, mol1)
    mol2 = OEMol(c_mol)
    oechem.OETriposAtomTypeNames(mol2)
    oechem.OEWriteConstMolecule(ofsTri, mol2)

ofsff.close()
ofsTri.close()